# Awari - Data Science

## Projeto - Python com SQLite

## 1. Considerações iniciais

Para que você assimile melhor os comandos do SQL, preparamos este projeto de Python com SQLite. Neste notebook Jupyter, você irá reforçando alguns conceitos da unidade atual e deverá executar algumas tarefas pelo caminho. Todo o projeto deve ser executado neste Jupyter Notebook. Por fim, se desejar, revise as respostas com o seu mentor.

### 1.1. Por que SQLite?
O SQLite é uma pequena biblioteca em C que possui um banco de dados SQL embutido. Sem o apoio de um poderoso SGDB (Sistema de Gerenciamento de Banco de Dados) como nas soluções do mercado, o SQLite é ideal para fins de aprendizagem, pois sua instalação é fácil e não requer nenhuma configuração inicial. Deste modo, podemos focar nos comandos do SQL.

### 1.2. Prepare seu ambiente
Baixe e instale o SQLite antes de começar. Resumidamente, você irá acessar a página de [Downloads](https://www.sqlite.org/download.html) do SQLite e baixar os executáveis para a mesma pasta deste notebook. Caso você encontre dificuldades:

- Assista a este [vídeo](https://www.youtube.com/watch?v=VcKKt6OTlJc) para "instalar" o SQLite no Windows ou;
- Ou siga este [manual](http://www.devfuria.com.br/linux/instalando-sqlite/) para as plataformas Linux.

### 1.3. Conjunto de dados

O conjunto de dados que iremos utilizar é derivado do [Titanic](https://www.kaggle.com/c/titanic/data). Na verdade, dividimos o conjunto em dois arquivos CSV - [passageiros](passageiros.csv) e [passagens](passagens.csv). Cada um destes arquivo serão inseridos dentro do SQLite como tabelas, onde você executará suas consultas SQL.

#### 1.3.1. Descrição do dados

No arquivo [passageiros](passageiros.csv), você encontrará:
- PassengerId: Id do passageiro
- Survived: Se o passageiro sobreviveu ou não. Se 0, então não sobreviveu. Se 1, sobreviveu.
- Pclass: Classe em que o passageiro estava viajando. Se 1, então estava na primeira classe e assim por diante.
- Name: Nome do passageiro.
- Sex: Gênero do passageiro. Masculino ou feminino.
- Age: Idade.
- Sibsp: número de irmãos / esposas a bordo do navio.
- Parch: número de pais / filhos a bordo do navio.

E no arquivo [passagens](passagens.csv):
- PassengerId: Id do passageiro
- Ticket: número do ticket de embarque
- Fare: valor pago pelo ticket.
- Cabin: cabine do passageiro.
- Embarked: porto de embarque do passageiro. C = Cherbourg, Q = Queenstown e S = Southampton


## 2. Procedimentos

### 2.1. Leitura dos arquivos CSV
Antes de importarmos os dados dentro das tabelas, precisamos começar lendo os arquivos CSV. O pequeno código abaixo importa os dados do CSV em uma lista de dicionários em Python.

In [1]:
import csv 

def importar_csv(arquivo):
    dados = []
    with open(arquivo) as arq:
        leitor = csv.DictReader(arq)
        for l in leitor:
            dados.append(dict(l))
    
    print('O conjunto de dados tem %s registros.' % len(dados))
    
    return dados

### TAREFA 01

1. Importe o arquivo passagens.csv usando o mesmo código anterior.
2. Atualize as partes contendo `...` com o que falta no seu código

**DICA**: Não reinvente a roda, reutilize a função anterior.

In [2]:
# Insira seu código aqui
passagens = importar_csv('passagens.csv')

O conjunto de dados tem 891 registros.


### TAREFA 02
1. Importe o arquivo passageiros.csv usando o mesmo código anterior.

In [3]:
# Insira seu código aqui
passageiros = importar_csv('passageiros.csv')


O conjunto de dados tem 891 registros.


### 2.2. Criando o banco de dados
Para manipular um banco SQLite com Python, temos que utilizar uma biblioteca específica.

In [4]:
import sqlite3
conn = sqlite3.connect('titanic.db')

O comando acima cria o arquivo [titanic.db](titanic.db), que armazará nossos dados.

### 2.3. Criando as tabelas
Neste momento, temos do dados e o banco SQLlite. Então é hora de começar à usar comandos SQL para criar as tabelas que armazenarão nossos dados dentro do banco.

No SQLlite, precisamos primeiro criar uma conexão e um cursor para executar nossos comandos no banco. Para facilicar, o código abaixo facilita este processo. Você só precisa passar seu comando SQL para a função.

In [5]:
def executa(comando):
    cursor = conn.cursor()
    cursor.execute(comando)
    cursor.close()

Não execute o comando abaixo, pois ele dá um erro. É só um exemplo

In [ ]:
# Exemplo
#comando = '<SEU_COMANDO_AQUI>'
#executa(comando)

### TAREFA 03
1. Cria a tabela passageiros.
2. PassengerId deve ser a chave primária.
3. Atualize as partes contendo `...` com o que falta no seu código

In [6]:
# Insira seu código aqui
tabela_passageiros_comando = """
CREATE TABLE passageiros(PassengerId INTEGER NOT NULL PRIMARY KEY, Survived INTEGER NOT NULL, Pclass INTEGER NOT NULL, Name VARCHAR(200), Sex VARCHAR(20), Age INTEGER, SibSp INTEGER, Parch INTEGER);
"""

In [7]:
executa(tabela_passageiros_comando)

### TAREFA 04
1. Cria a tabela passagens.
2. PassengerId deve ser a chave primária.
3. Atualize as partes contendo `...` com o que falta no seu código

In [8]:
# Insira seu código aqui
tabela_passagens_comando = """
CREATE TABLE passagens(PassengerId INTEGER NOT NULL PRIMARY KEY, Ticket VARCHAR(200), Fare DECIMAL(9,2), Cabin VARCHAR(20), Embarked VARCHAR(5));
"""


In [9]:
executa(tabela_passagens_comando)

**Atenção:** Caso você tente executar os comandos de criação de tabelas mais de uma vez, perceberá um erro de operação informando que a tabela já existe.

Vamos verificar se as tabelas foram realmente criadas? Abra o terminal e na pasta atual digite:
```
$ sqlite3 titanic.db ".tables"
```
O terminal deve retornar com as tabelas existentes em seu banco [titanic.db](titanic.db). Neste casos, *passageiros* e *passagens*. Lembre-se que neste momento, não temos nenhum dado inseridos nas tabelas.

Caso queira verifcar mais algumas informações das tabelas criadas, pode executar esta outra instrução:
```
$ sqlite3 titanic.db "PRAGMA table_info(passageiros)"
```

### TAREFA 05
1. Pelo terminal, verifique mais informações da tabela passagens.

### 2.4. Inserindo dados nas tabelas
Usando o comando INSERT INTO, vamos popular as nossas tabelas. O processo é simples, percorremos as listas de dicionários, montamos nosso comando INSERT INTO e executamos - uma a uma. Para a tabela passageiros:

In [10]:
for p in passageiros:
    p['Name'] = p['Name'].replace('"', "'")
    inserir_passageiro = ("""
    INSERT INTO passageiros(PassengerId, Survived, Pclass, Name, Sex, Age, Sibsp, Parch)
    VALUES (%s, "%s", "%s", "%s", "%s", "%s", "%s", "%s")""") % (p['PassengerId'], p['Survived'], p['Pclass'],
                                                                 p['Name'], p['Sex'], p['Age'], p['SibSp'],
                                                                 p['Parch'])
    executa(inserir_passageiro)

### TAREFA 06
1. Insira os dados de passagens dentro da tabela passagens

In [11]:
# Insira seu código aqui
for p in passagens:
    inserir_passagens = ("""
    INSERT INTO passagens(PassengerId, Ticket, Fare, Cabin, Embarked)
    VALUES (%s, "%s", "%s", "%s", "%s")""") % (p['PassengerId'], p['Ticket'], p['Fare'], 
                                               p['Cabin'], p['Embarked'])
    executa(inserir_passagens)

### 2.5. Executando consultas
Agora que os dados estão inseridos no arquivo banco SQLite, vamos executar alguns consultas. Vamos começar fazendo uma seleção de todas as colunas das tabelas.

O comando SELECT retorna dados, deste modo, precisamos fazer alguma alteração na função executa. utilize o *executa_consulta()*.

In [12]:
def executa_consulta(consulta):
    cursor = conn.cursor()
    cursor.execute(consulta)
    for linha in cursor.fetchall():
        print(linha)
    
    cursor.close()

Testando nosso código. Repare que o comando LIMIT foi utilizado para limitar a quantidade de linhas.

In [13]:
consulta_passageiro = """SELECT * FROM PASSAGEIROS LIMIT 10"""
executa_consulta(consulta_passageiro)

(1, 0, 3, 'Braund, Mr. Owen Harris', 'male', 22, 1, 0)
(2, 1, 1, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'female', 38, 1, 0)
(3, 1, 3, 'Heikkinen, Miss. Laina', 'female', 26, 0, 0)
(4, 1, 1, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'female', 35, 1, 0)
(5, 0, 3, 'Allen, Mr. William Henry', 'male', 35, 0, 0)
(6, 0, 3, 'Moran, Mr. James', 'male', '', 0, 0)
(7, 0, 1, 'McCarthy, Mr. Timothy J', 'male', 54, 0, 0)
(8, 0, 3, 'Palsson, Master. Gosta Leonard', 'male', 2, 3, 1)
(9, 1, 3, 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', 'female', 27, 0, 2)
(10, 1, 2, 'Nasser, Mrs. Nicholas (Adele Achem)', 'female', 14, 1, 0)


In [ ]:
consulta_passagens = """SELECT * FROM PASSAGENS LIMIT 10"""
executa_consulta(consulta_passagens)

(1, 'A/5 21171', 7.25, '', 'S')
(2, 'PC 17599', 71.2833, 'C85', 'C')
(3, 'STON/O2. 3101282', 7925, '', 'S')
(4, '113803', 53.1, 'C123', 'S')
(5, '373450', 8.05, '', 'S')
(6, '330877', 8.4583, '', 'Q')
(7, '17463', 51.8625, 'E46', 'S')
(8, '349909', 21075, '', 'S')
(9, '347742', 11.1333, '', 'S')
(10, '237736', 30.0708, '', 'C')


Também podemos imprimir mais informações sobre uma tabela usando o seguinte comando:

In [14]:
executa_consulta('pragma table_info(passageiros)')

(0, 'PassengerId', 'INTEGER', 1, None, 1)
(1, 'Survived', 'INTEGER', 1, None, 0)
(2, 'Pclass', 'INTEGER', 1, None, 0)
(3, 'Name', 'VARCHAR(200)', 0, None, 0)
(4, 'Sex', 'VARCHAR(20)', 0, None, 0)
(5, 'Age', 'INTEGER', 0, None, 0)
(6, 'SibSp', 'INTEGER', 0, None, 0)
(7, 'Parch', 'INTEGER', 0, None, 0)


### TAREFA 07
1. Verifique quantos tipos de classe (PClass) existem na tabela passageiros.

In [15]:
# Insira seu código aqui
consulta = """SELECT DISTINCT(PClass) FROM PASSAGEIROS"""
executa_consulta(consulta)

(3,)
(1,)
(2,)


### TAREFA 08
1. Selecione os passageiros do sexo masculino (Sex) e que estejam na segunda classe (PClass).

**DICA**: Use a cláusula WHERE e uma condicional (AND ou OR).

In [16]:
# Insira seu código aqui
consulta = """SELECT * FROM PASSAGEIROS WHERE Sex='male' and PClass= 2"""
executa_consulta(consulta)

(18, 1, 2, 'Williams, Mr. Charles Eugene', 'male', '', 0, 0)
(21, 0, 2, 'Fynney, Mr. Joseph J', 'male', 35, 0, 0)
(22, 1, 2, 'Beesley, Mr. Lawrence', 'male', 34, 0, 0)
(34, 0, 2, 'Wheadon, Mr. Edward H', 'male', 66, 0, 0)
(71, 0, 2, 'Jenkin, Mr. Stephen Curnow', 'male', 32, 0, 0)
(73, 0, 2, 'Hood, Mr. Ambrose Jr', 'male', 21, 0, 0)
(79, 1, 2, 'Caldwell, Master. Alden Gates', 'male', 0.83, 0, 2)
(100, 0, 2, 'Kantor, Mr. Sinai', 'male', 34, 1, 0)
(118, 0, 2, 'Turpin, Mr. William John Robert', 'male', 29, 1, 0)
(121, 0, 2, 'Hickman, Mr. Stanley George', 'male', 21, 2, 0)
(123, 0, 2, 'Nasser, Mr. Nicholas', 'male', 32.5, 1, 0)
(135, 0, 2, 'Sobey, Mr. Samuel James Hayden', 'male', 25, 0, 0)
(136, 0, 2, 'Richard, Mr. Emile', 'male', 23, 0, 0)
(145, 0, 2, 'Andrew, Mr. Edgardo Samuel', 'male', 18, 0, 0)
(146, 0, 2, 'Nicholls, Mr. Joseph Charles', 'male', 19, 1, 1)
(149, 0, 2, "Navratil, Mr. Michel ('Louis M Hoffman')", 'male', 36.5, 0, 2)
(150, 0, 2, 'Byles, Rev. Thomas Roussel Davids', 'male'

### TAREFA 09
1. Conte quantos passageiros existem por sexo.

**DICA**: Use GROUP BY e COUNT().

In [17]:
# Insira seu código aqui
consulta = """SELECT Sex, count() FROM PASSAGEIROS GROUP BY Sex"""
executa_consulta(consulta)

('female', 314)
('male', 577)


### TAREFA 10
1. Verifique o valor médio das passagens em cada porto de embarque

**DICA**: Use GROUP BY e AVG(). A presença de valores missing pode apresentar alguns resultados estranhos.

In [20]:
# Insira seu código aqui
consulta = """SELECT AVG(Fare), Embarked FROM PASSAGENS GROUP BY Embarked"""
executa_consulta(consulta)

(80.0, '')
(12709.34566190476, 'C')
(2002.8429129870128, 'Q')
(2015.3069857142902, 'S')


### TAREFA 11
1. Selecione o nome (Name), sexo(Sex) e tarifa(Fare) paga de 5 passageiros
2. Utilize as tabelas passageiros e passagens.

**DICA**: Use JOIN.

In [21]:
# Insira seu código aqui
# Insira seu código aqui
consulta = """SELECT Name, Sex, Fare FROM PASSAGEIROS INNER JOIN PASSAGENS 
ON passageiros.PassengerId == passagens.PassengerId LIMIT 5"""
executa_consulta(consulta)

('Braund, Mr. Owen Harris', 'male', 7.25)
('Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'female', 71.2833)
('Heikkinen, Miss. Laina', 'female', 7925)
('Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'female', 53.1)
('Allen, Mr. William Henry', 'male', 8.05)


### TAREFA 12
1. Selecione o valor máximo (Fare) das passagens por classe.

In [ ]:
# Insira seu código aqui
consulta = """SELECT MAX(Fare) FROM PASSAGENS GROUP BY PClass"""
executa_consulta(consulta)

OperationalError: ignored

### TAREFA 13
1. Descubra quantas pessoas embarcaram por porto
2. Agrupando por porto e sexo.

In [ ]:
# Insira seu código aqui
consulta = """..."""
executa_consulta(consulta)

## 3. Conclusão

A quantidade de consultas e a lista de comandos SQL é enorme. A interação foi somente relembrar alguns comandos básicos para que você se sinta confortável para explorar qualquer banco de dados SQL que você encontrar.


### Awari -  <a href="https://awari.com.br/"> awari.com.br</a>